In [1]:
import pickle
import gc

with open('./backups/reverse_index_20251103_143918.pkl', 'rb') as f:
    result = pickle.load(f)


In [2]:
from tqdm import tqdm

def merge_pickles(file_paths, output_path):
    global result
    for file_path in tqdm(file_paths):
        with open(file_path, 'rb') as f:
            current = pickle.load(f)
        
        for key, value in tqdm(current.items()):
            if key in result:
                result[key].extend(value)
            else:
                result[key] = value
        
        del current
        gc.collect()
    
    with open(output_path, 'wb') as f:
        pickle.dump(result, f, protocol=pickle.HIGHEST_PROTOCOL)
    
    del result
    gc.collect()

files = ['./backups/reverse_index_20251103_153037.pkl', './backups/reverse_index_20251103_153014.pkl', './backups/reverse_index_20251103_153307.pkl']
merge_pickles(files, 'merged.pkl')


100%|██████████| 3/3 [07:22<00:00, 147.55s/it]


In [ ]:
import numpy as np
import cupy as cp
from tqdm import tqdm
import gc

def sort_dict_cuda_inplace(data_dict):
    keys_list = list(data_dict.keys())
    
    for key in tqdm(keys_list, desc="Sorting arrays"):
        arr = data_dict[key]
        
        if len(arr) == 0:
            continue
            
        values = np.array([x[1] for x in arr], dtype=np.float32)
        
        gpu_values = cp.asarray(values)
        gpu_indices = cp.argsort(-gpu_values)
        sorted_indices = cp.asnumpy(gpu_indices)
        
        sorted_arr = [arr[i] for i in sorted_indices]
        
        del data_dict[key]
        gc.collect()
        
        data_dict[key] = sorted_arr
        
        del gpu_values, gpu_indices, values, sorted_indices, arr
        cp.get_default_memory_pool().free_all_blocks()
    
    return data_dict


data = {
    'key1': [('a', 0.7), ('b', 1.5), ('c', 0.3)],
    'key2': [('x', 6.1), ('y', 4.3), ('z', 8.9), ('w', 2.1)],
    'key3': [('p', 3.2), ('q', 1.1)]
}

sorted_data = sort_dict_cuda_inplace(data)

for key, val in sorted_data.items():
    print(f"{key}: {val}")

[0;31mKernelOutOfMemory[0m: Kernel ran out of memory and has been restarted. If the restart fails, restart the kernel from the Kernel menu.
If the error persists, try choosing a different configuration or optimizing your code.